# 정신건강 보이스 챗봇

1. 사용자 입력을 음성으로 받는다.
2. STT를 적용하여 텍스트로 변환한다.
3. 변환된 텍스트를 입력으로 하여 프롬프트 엔지니어링을 해 api요청을 보낸다.
4. 반환받은 응답을 TTS를 적용하여 음성으로 재생한다.
(+) 2에서 입력된 텍스트와 4에서 반환된 응답을 채팅 내역 보듯이 (카카오톡 대화처럼) 현출되도록 출력한다.

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
#! pip install pyttsx3

### TTS

In [3]:
import pyttsx3
def pick_korean_voice(engine: pyttsx3.Engine):
    try:
        for v in engine.getProperty("Voices"):
            name = (getattr(v, "name", "") or "").lower()
            langs = [str(x).lower() for x in getattr(v, "languages", [])]
            if "ko" in langs:
                return v.id
    
    except Exception:
        pass
    return None

engine = pyttsx3.init()
engine.setProperty("rate", 175)
ko_voice_id = pick_korean_voice(engine)
if ko_voice_id:
    engine.setProperty("voice", ko_voice_id)

def speak(text= str):
    engine.say(text)
    engine.runAndWait()

### 위험도 평가

In [4]:
RISK_LEXICON = {
    "selfHarmCore": [
        "죽고 싶", "살고 싶지 않", "사라지고 싶", "없어지고 싶", "극단적 선택", "자살", "목숨을 끊",
        "생을 마감", "모든 걸 끝내", "더 살 의미가 없", "유서", "번개탄", "연탄", "자해", "앞이 보이지"
    ],
    "planDetails": ["약을 많이", "과다 복용", "뛰어내", "베어", "목을", "칼로", "피를", "지하철", "다리에서"],
    "timeUrgency": ["지금", "곧", "오늘", "방금", "당장"],
    "hopelessness": ["절망", "희망이 없", "끝났", "포기", "나는 짐", "무가치"],
    "otherHarm": ["죽여", "해칠", "폭탄", "터뜨리", "불 지르"],
}

def assess_risk(text = str):
    t = text.lower()
    score = 0
    signals = []

    def hit(arr, w):
        nonlocal score, signals
        for k in arr:
            if k in t:
                score += w
                signals.append(k)
    
    hit(RISK_LEXICON["selfHarmCore"], 2)
    hit(RISK_LEXICON["planDetails"], 3)
    hit(RISK_LEXICON["timeUrgency"], 2)
    hit(RISK_LEXICON["hopelessness"], 1)
    hit(RISK_LEXICON["otherHarm"], 2)

    level = "LOW"
    if score >= 5:
        level = "high"
    elif score >=3:
        level = "Medium"
    return {"level": level, "score": score, "signals": signals}

### Chat Completions

In [5]:
# pip install openai
import os
from openai import OpenAI

def build_system_prompt() -> str:
    return (
        "너는 공감적이고 간결한 한국어 보이스 어시스턴트다. "
        "의료 조언이나 진단을 대체하지 않는다. "
        "응답은 80단어 내외로 유지하고, 사용자의 감정을 1문장으로 인정한 뒤 "
        "도움이 되는 아주 작은 다음 행동 1가지를 제안한다. "
        "위험 신호(자/타해 암시/계획/절망)가 보이면 사용자의 동의를 전제로 "
        "호흡/그라운딩 등 간단한 안정화와, 신뢰하는 사람에게 연락을 고려하도록 부드럽게 권유한다."
    )

def call_llm(query, temperature= 0.3):
    # API 키가 없으면 안전한 기본 응답 반환
    if not os.getenv("OPENAI_API_KEY"):
        return (
            "당신의 마음을 소중히 느끼고 있어요. 지금은 짧게 호흡을 고르고 "
            "가장 가벼운 일 하나만 해보는 건 어떨까요? 원하신다면 믿을 수 있는 분께 "
            "잠시 함께 있어 달라고 부탁해 보는 것도 도움이 돼요."
        )

    try:
        client = OpenAI()  # OPENAI_API_KEY는 환경변수로 설정
        system_instruction = build_system_prompt()
        user_message = (
            "사용자의 발화를 바탕으로 공감과 간결한 제안을 해 주세요.\n"
            f"사용자 발화: {query}"
        )

        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_instruction},
                {"role": "user", "content": user_message},
            ],
            temperature=temperature,
            max_tokens=700,
            top_p=1.0,
            frequency_penalty=0,
            presence_penalty=0,
        )
        return resp.choices[0].message.content.strip()

    except Exception:
        # 모델 호출 실패 시에도 안전한 기본 응답
        return (
            "지금 필요한 도움을 함께 찾아볼게요. 잠시 자리에서 일어나 물 한 잔을 마시고, "
            "어깨와 턱 힘을 살짝 풀어보세요. 원하시면 믿을 수 있는 분께 연락해 보세요."
        )

In [6]:
# !pip install SpeechRecognition
# !pip install pydub pyaudio

In [7]:
import speech_recognition as sr

recognizer = sr.Recognizer()
recognizer.pause_threshold = 0.7
recognizer.energy_threshold = 300
PHRASE_LIMIT = 20

print("정신건강 보이스 챗봇을 시작합니다. '종료'라고 말하면 끝납니다.\n")

try:
    while True:
        with sr.Microphone() as source:
            print("말씀하세요… (소음 보정 중)")
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            print("듣는 중…")
            audio = recognizer.listen(source, phrase_time_limit=PHRASE_LIMIT)

        try:
            query = recognizer.recognize_google(audio, language="ko-KR").strip()
            print(f"[사용자] {query}")

            # 종료
            if query == "종료":
                speak("종료합니다. 수고하셨어요")
                break

            # 위험 평가
            risk = assess_risk(query)
            print(f"[리스크] level={risk['level']} score={risk['score']} signals={risk['signals']}")

            # high 일 때 안정화 멘트
            if risk["level"] == "high":
                safety_msg = (
                    "지금은 당신의 안전이 가장 중요해요. 원하신다면 호흡을 천천히 다섯 번 해볼까요? "
                    "가능하면 믿을 수 있는 분께 오늘 같이 있어 달라고 부탁해 보세요. "
                    "저는 차분히 계속 들어드릴게요."
                )
                print(f"[봇:안정화] {safety_msg}")
                speak(safety_msg)
            
            # LLM 응답
            replay =call_llm(query)
            print(f"[봇] {replay}")
            speak(replay)


        except sr.UnknownValueError:
            print("[알림] 인식 실패: 다시 말씀해 주세요.")
            speak("죄송해요, 잘 듣지 못했어요. 다시 말씀해 주세요.")
        except sr.RequestError as e:
            print(f"[오류] 음성 인식 요청 실패: {e}")
            speak("현재 음성 인식 서버에 연결하기 어렵습니다.")
        except Exception as e:
            print(f"[오류] 예외 발생: {e}")
            speak("문제가 발생했어요. 잠시 후 다시 시도해 주세요.")

except KeyboardInterrupt:
    print("\n사용자 중단으로 종료합니다.")
finally:
    try:
        engine.stop()
    except Exception:
        pass
    print("프로그램을 종료합니다.")


정신건강 보이스 챗봇을 시작합니다. '종료'라고 말하면 끝납니다.

말씀하세요… (소음 보정 중)
듣는 중…
[사용자] 더 이상 삶을 살기가 싫어
[리스크] level=LOW score=0 signals=[]
[봇] 당신이 힘든 감정을 느끼고 있다는 것을 이해합니다. 이런 기분이 드는 것은 정말 힘든 일입니다. 지금은 믿을 수 있는 친구나 가족에게 이야기해보는 것이 좋을 것 같아요. 그들과 함께 이야기를 나누면 조금이나마 마음이 편해질 수 있습니다.
말씀하세요… (소음 보정 중)
듣는 중…
[알림] 인식 실패: 다시 말씀해 주세요.
말씀하세요… (소음 보정 중)
듣는 중…
[알림] 인식 실패: 다시 말씀해 주세요.
말씀하세요… (소음 보정 중)
듣는 중…
[알림] 인식 실패: 다시 말씀해 주세요.
말씀하세요… (소음 보정 중)
듣는 중…
[알림] 인식 실패: 다시 말씀해 주세요.
말씀하세요… (소음 보정 중)
듣는 중…
[알림] 인식 실패: 다시 말씀해 주세요.
말씀하세요… (소음 보정 중)
듣는 중…
[알림] 인식 실패: 다시 말씀해 주세요.
말씀하세요… (소음 보정 중)
듣는 중…
[사용자] 여기에다가 가까이
[리스크] level=LOW score=0 signals=[]
[봇] 당신의 마음이 불안한 것 같네요. 가까이에서 뭔가를 느끼고 싶으신가요? 주변의 안전한 공간에서 잠시 깊게 숨을 쉬어보는 것도 좋을 것 같아요.
말씀하세요… (소음 보정 중)
듣는 중…
[알림] 인식 실패: 다시 말씀해 주세요.
말씀하세요… (소음 보정 중)
듣는 중…
[사용자] 어지럽다
[리스크] level=LOW score=0 signals=[]
[봇] 어지럽다고 하니 불편하고 힘드실 것 같아요. 잠시 앉아서 깊게 숨을 쉬어보는 건 어떨까요? 천천히 숨을 들이쉬고 내쉬면서 몸을 안정시켜보세요. 필요하다면 가까운 사람에게 이야기해보는 것도 좋습니다.
말씀하세요… (소음 보정 중)
듣는 중…
[알림] 인식 실패: 다시 말씀해 주세요.
말씀하세요… (소음 보정 중)
듣는 중…
[알림] 